In [60]:
import pandas as pd
import numpy as np
from pathlib import Path

In [61]:
resultsdir = Path('../3dA/run_1/results')
expr_path = Path(resultsdir/'trans_expr.csv')
gtf_path = Path('/home/chlab/flync_retest/dlinct/3dA/run_1/cuffcompare/cuffcomp.gtf.tracking')
dge_path = Path(resultsdir/'dge.csv')
mlc_path = Path(resultsdir/'new-lincRNAs-ml.csv')
bed_path = Path(resultsdir/'non-coding/new-lincRNAs.bed')

In [63]:
expr = pd.read_csv(expr_path)
expr = pd.merge(expr['t_name'], expr.filter(axis=1, regex='FPKM'), left_index=True, right_index=True)
expr = pd.merge(expr.iloc[:, [0, -2, -1]], expr.iloc[:, 1:len(expr.columns)-2], left_index=True, right_index=True)
expr

,t_name,mean.FPKM,sd.FPKM,FPKM.SRR18432803,FPKM.SRR18432804,FPKM.SRR18432805,FPKM.SRR18432812,FPKM.SRR18432813,FPKM.SRR18432814
0,FBtr0306592,0.819475,1.521779,0.032063,0.000000,0.000000,0.000000,4.173363,0.711423
1,MSTRG.2.13,1.308683,1.163723,2.878206,0.936465,1.280486,2.756943,0.000000,0.000000
2,FBti0059810-RA,0.990779,1.067957,0.250316,0.084805,0.079435,0.904550,3.068011,1.557557
3,MSTRG.2.15,0.456273,1.020257,0.000000,0.000000,2.737637,0.000000,0.000000,0.000000
4,FBtr0078164,3.262830,2.180900,5.513323,3.011719,2.510013,6.531014,0.012749,1.998161
...,...,...,...,...,...,...,...,...,...
31579,FBtr0100885,3.116718,1.550686,3.565916,5.440633,4.183465,2.765361,2.232458,0.512474
31580,FBtr0433499,112.806228,46.214093,122.844162,162.778488,74.349846,170.093124,107.648514,39.123234
31581,FBtr0100887,8.227221,3.148608,11.097774,6.694445,7.967892,9.593075,11.694038,2.316104
31582,FBtr0100888,44439.697103,18538.068722,33898.117188,43216.261719,44693.273438,58515.539062,72719.773438,13595.217773


In [64]:
gtf = pd.read_csv(gtf_path, delimiter='\t', header=None)
gtf = pd.merge(gtf, (gtf[4].str.split('|', expand=True)), left_index=True, right_index=True)
gtf.drop(columns=['2_x', '3_x', '4_x', '0_y', '2_y', '3_y', '4_y', 5, 6, 7], inplace=True)
gtf.rename(columns={'0_x': 'transcript', '1_x': 'locus', '1_y': 'transfrag'}, inplace=True)
gtf = gtf[gtf['transfrag'].str.contains('MSTRG')]
gtf

,transcript,locus,transfrag
2,TCONS_00000003,XLOC_000003,MSTRG.4.1
10,TCONS_00000011,XLOC_000010,MSTRG.13.1
12,TCONS_00000013,XLOC_000010,MSTRG.13.2
17,TCONS_00000018,XLOC_000010,MSTRG.13.9
22,TCONS_00000023,XLOC_000011,MSTRG.15.5
...,...,...,...
57123,TCONS_00057124,XLOC_028872,MSTRG.25376.1
57124,TCONS_00057125,XLOC_028873,MSTRG.25377.1
57125,TCONS_00057126,XLOC_028874,MSTRG.25378.1
57126,TCONS_00057127,XLOC_028875,MSTRG.25379.1


In [65]:
dge = pd.read_csv(dge_path)
dge

,geneNames,geneIDs,feature,id,fc,pval,qval
0,l(2)gl,MSTRG.2,transcript,13,2.074349,0.660092,0.994009
1,.,MSTRG.2,transcript,15,0.639370,0.804809,0.996096
2,.,MSTRG.2,transcript,16,0.862867,0.843103,0.996096
3,.,MSTRG.2,transcript,17,0.178912,0.143501,0.994009
4,Cda5,MSTRG.7,transcript,27,0.445109,0.719353,0.994712
...,...,...,...,...,...,...,...
31579,mt:tRNA:Ser-TGA,MSTRG.25411,transcript,60045,2.583957,0.438914,0.994009
31580,mt:ND1,MSTRG.25412,transcript,60046,3.491857,0.498981,0.994009
31581,mt:tRNA:Leu-TAG,MSTRG.25413,transcript,60047,1.726371,0.751381,0.995588
31582,mt:lrRNA,MSTRG.25414,transcript,60048,1.448893,0.858362,0.996096


In [66]:
bed = pd.read_csv(bed_path, delimiter='\t', header=None)
bed.drop(columns=[4, 5], inplace=True)
bed.rename(columns={0:'chr', 1:'start', 2:'end', 3:'name'}, inplace=True)
bed

,chr,start,end,name
0,2L,77805,78230,TCONS_00004626.1
1,2L,78724,79021,TCONS_00004626.2
2,2L,78742,78869,TCONS_00004627.1
3,2L,78938,79021,TCONS_00004627.2
4,2L,90157,90847,TCONS_00008934.1
...,...,...,...,...
6166,Y,3629695,3630983,TCONS_00056960.2
6167,Y,3631152,3631967,TCONS_00057124.1
6168,Y,3633207,3633814,TCONS_00057125.1
6169,Y,3637823,3640009,TCONS_00057126.1


In [67]:
mlc = pd.read_csv(mlc_path)
mlc.drop(columns=['lncRNA', 'Prob_False'], inplace=True)
mlc

,name,Prob_True
0,TCONS_00008935.1,0.62
1,TCONS_00008949.1,0.51
2,TCONS_00008960.1,0.51
3,TCONS_00008980.1,0.74
4,TCONS_00008982.1,0.72
...,...,...
575,TCONS_00047779.2,0.58
576,TCONS_00056262.1,0.57
577,TCONS_00056510.3,0.55
578,TCONS_00029301.2,0.62


In [68]:
final = pd.merge(mlc, bed, on='name')
final = pd.merge(final, final['name'].str.split('.', expand=True), left_index=True, right_index=True)
final.rename(columns={0: 'transcript', 1: 'exon'}, inplace=True)
final.drop(columns='name', inplace=True)
final

,Prob_True,chr,start,end,transcript,exon
0,0.62,2L,365866,367076,TCONS_00008935,1
1,0.51,2L,634834,635975,TCONS_00008949,1
2,0.51,2L,764810,766957,TCONS_00008960,1
3,0.74,2L,1478713,1479834,TCONS_00008980,1
4,0.72,2L,1485716,1486325,TCONS_00008982,1
...,...,...,...,...,...,...
570,0.58,X,20729339,20729892,TCONS_00056418,1
571,0.68,X,20754581,20755037,TCONS_00056420,1
572,0.78,X,20757073,20757525,TCONS_00056421,1
573,0.60,X,20982233,20983671,TCONS_00056427,1


In [69]:
final = pd.merge(final, gtf, on='transcript')
final

,Prob_True,chr,start,end,transcript,exon,locus,transfrag
0,0.62,2L,365866,367076,TCONS_00008935,1,XLOC_004290,MSTRG.83.1
1,0.51,2L,634834,635975,TCONS_00008949,1,XLOC_004304,MSTRG.137.1
2,0.51,2L,764810,766957,TCONS_00008960,1,XLOC_004315,MSTRG.155.1
3,0.74,2L,1478713,1479834,TCONS_00008980,1,XLOC_004335,MSTRG.267.1
4,0.72,2L,1485716,1486325,TCONS_00008982,1,XLOC_004337,MSTRG.269.1
...,...,...,...,...,...,...,...,...
570,0.58,X,20729339,20729892,TCONS_00056418,1,XLOC_028293,MSTRG.24412.1
571,0.68,X,20754581,20755037,TCONS_00056420,1,XLOC_028295,MSTRG.24414.1
572,0.78,X,20757073,20757525,TCONS_00056421,1,XLOC_028296,MSTRG.24415.1
573,0.60,X,20982233,20983671,TCONS_00056427,1,XLOC_028302,MSTRG.24441.1


In [70]:
df = final['transfrag'].str.split('.', -1, expand=True)
df['idx'] = df[[0, 1]].agg('.'.join, axis=1)
df = df['idx']
final['idx'] = df
final = pd.merge(final, dge[['geneIDs', 'fc', 'qval']], left_on='idx', right_on='geneIDs', how='left')
final.drop(columns=['idx', 'geneIDs'], inplace=True)
final


,Prob_True,chr,start,end,transcript,exon,locus,transfrag,fc,qval
0,0.62,2L,365866,367076,TCONS_00008935,1,XLOC_004290,MSTRG.83.1,0.934072,0.999074
1,0.51,2L,634834,635975,TCONS_00008949,1,XLOC_004304,MSTRG.137.1,2.021739,0.994009
2,0.51,2L,764810,766957,TCONS_00008960,1,XLOC_004315,MSTRG.155.1,1.941763,0.994009
3,0.74,2L,1478713,1479834,TCONS_00008980,1,XLOC_004335,MSTRG.267.1,0.915108,0.996096
4,0.72,2L,1485716,1486325,TCONS_00008982,1,XLOC_004337,MSTRG.269.1,9.938186,0.994009
...,...,...,...,...,...,...,...,...,...,...
574,0.58,X,20729339,20729892,TCONS_00056418,1,XLOC_028293,MSTRG.24412.1,0.451115,0.994009
575,0.68,X,20754581,20755037,TCONS_00056420,1,XLOC_028295,MSTRG.24414.1,3.703429,0.994009
576,0.78,X,20757073,20757525,TCONS_00056421,1,XLOC_028296,MSTRG.24415.1,0.700523,0.996096
577,0.60,X,20982233,20983671,TCONS_00056427,1,XLOC_028302,MSTRG.24441.1,1.077881,0.999074


In [71]:
final = pd.merge(final, expr, left_on='transfrag', right_on='t_name', how='left')
final

,Prob_True,chr,start,end,transcript,exon,locus,transfrag,fc,qval,t_name,mean.FPKM,sd.FPKM,FPKM.SRR18432803,FPKM.SRR18432804,FPKM.SRR18432805,FPKM.SRR18432812,FPKM.SRR18432813,FPKM.SRR18432814
0,0.62,2L,365866,367076,TCONS_00008935,1,XLOC_004290,MSTRG.83.1,0.934072,0.999074,MSTRG.83.1,1.846249,1.818760,3.723752,1.337980,4.912979,0.491755,0.501407,0.109620
1,0.51,2L,634834,635975,TCONS_00008949,1,XLOC_004304,MSTRG.137.1,2.021739,0.994009,MSTRG.137.1,1.375609,1.264469,2.119220,1.807854,3.566761,0.082241,0.677575,0.000000
2,0.51,2L,764810,766957,TCONS_00008960,1,XLOC_004315,MSTRG.155.1,1.941763,0.994009,MSTRG.155.1,1.153053,1.183216,2.891880,1.298481,2.511638,0.065560,0.121107,0.029654
3,0.74,2L,1478713,1479834,TCONS_00008980,1,XLOC_004335,MSTRG.267.1,0.915108,0.996096,MSTRG.267.1,0.795870,0.923328,1.561134,0.736043,2.436187,0.041854,0.000000,0.000000
4,0.72,2L,1485716,1486325,TCONS_00008982,1,XLOC_004337,MSTRG.269.1,9.938186,0.994009,MSTRG.269.1,1.019255,1.235772,2.046372,3.244831,0.824328,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574,0.58,X,20729339,20729892,TCONS_00056418,1,XLOC_028293,MSTRG.24412.1,0.451115,0.994009,MSTRG.24412.1,1.788911,2.109311,0.510108,2.750710,6.101959,0.414039,0.156731,0.799919
575,0.68,X,20754581,20755037,TCONS_00056420,1,XLOC_028295,MSTRG.24414.1,3.703429,0.994009,MSTRG.24414.1,1.917513,1.432749,1.452209,5.011845,2.018339,0.921912,0.950350,1.150422
576,0.78,X,20757073,20757525,TCONS_00056421,1,XLOC_028296,MSTRG.24415.1,0.700523,0.996096,MSTRG.24415.1,0.970802,1.342119,3.431592,0.101981,2.158372,0.132867,0.000000,0.000000
577,0.60,X,20982233,20983671,TCONS_00056427,1,XLOC_028302,MSTRG.24441.1,1.077881,0.999074,MSTRG.24441.1,1.716519,1.711570,1.519871,2.956121,4.868297,0.668217,0.240488,0.046120


In [72]:
final.drop(columns='t_name', inplace=True)
final

,Prob_True,chr,start,end,transcript,exon,locus,transfrag,fc,qval,mean.FPKM,sd.FPKM,FPKM.SRR18432803,FPKM.SRR18432804,FPKM.SRR18432805,FPKM.SRR18432812,FPKM.SRR18432813,FPKM.SRR18432814
0,0.62,2L,365866,367076,TCONS_00008935,1,XLOC_004290,MSTRG.83.1,0.934072,0.999074,1.846249,1.818760,3.723752,1.337980,4.912979,0.491755,0.501407,0.109620
1,0.51,2L,634834,635975,TCONS_00008949,1,XLOC_004304,MSTRG.137.1,2.021739,0.994009,1.375609,1.264469,2.119220,1.807854,3.566761,0.082241,0.677575,0.000000
2,0.51,2L,764810,766957,TCONS_00008960,1,XLOC_004315,MSTRG.155.1,1.941763,0.994009,1.153053,1.183216,2.891880,1.298481,2.511638,0.065560,0.121107,0.029654
3,0.74,2L,1478713,1479834,TCONS_00008980,1,XLOC_004335,MSTRG.267.1,0.915108,0.996096,0.795870,0.923328,1.561134,0.736043,2.436187,0.041854,0.000000,0.000000
4,0.72,2L,1485716,1486325,TCONS_00008982,1,XLOC_004337,MSTRG.269.1,9.938186,0.994009,1.019255,1.235772,2.046372,3.244831,0.824328,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574,0.58,X,20729339,20729892,TCONS_00056418,1,XLOC_028293,MSTRG.24412.1,0.451115,0.994009,1.788911,2.109311,0.510108,2.750710,6.101959,0.414039,0.156731,0.799919
575,0.68,X,20754581,20755037,TCONS_00056420,1,XLOC_028295,MSTRG.24414.1,3.703429,0.994009,1.917513,1.432749,1.452209,5.011845,2.018339,0.921912,0.950350,1.150422
576,0.78,X,20757073,20757525,TCONS_00056421,1,XLOC_028296,MSTRG.24415.1,0.700523,0.996096,0.970802,1.342119,3.431592,0.101981,2.158372,0.132867,0.000000,0.000000
577,0.60,X,20982233,20983671,TCONS_00056427,1,XLOC_028302,MSTRG.24441.1,1.077881,0.999074,1.716519,1.711570,1.519871,2.956121,4.868297,0.668217,0.240488,0.046120


In [73]:
final.isna().sum()
final.fillna(value=0, inplace=True)

In [76]:
top_20_prob = final.sort_values('Prob_True', ascending=False).iloc[0:20,:]
top_20_prob

,Prob_True,chr,start,end,transcript,exon,locus,transfrag,fc,qval,mean.FPKM,sd.FPKM,FPKM.SRR18432803,FPKM.SRR18432804,FPKM.SRR18432805,FPKM.SRR18432812,FPKM.SRR18432813,FPKM.SRR18432814
151,0.90,2R,13026340,13027051,TCONS_00021270,1,XLOC_010808,MSTRG.7411.1,0.937094,0.996096,1.592317,1.504234,1.954666,2.040248,4.518849,0.329949,0.243803,0.466386
533,0.90,X,11212364,11213061,TCONS_00055983,1,XLOC_027858,MSTRG.22778.1,0.768280,0.994009,1.915354,0.987895,2.351172,2.353038,3.626470,1.410254,0.621750,1.129441
494,0.89,X,4753265,4754217,TCONS_00055642,1,XLOC_027517,MSTRG.21699.1,1.057560,0.999074,2.235400,1.608259,3.365016,3.502656,4.433946,1.159168,0.000000,0.951611
486,0.88,X,4206331,4207482,TCONS_00055601,1,XLOC_027476,MSTRG.21600.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
373,0.86,3R,11510551,11511190,TCONS_00044738,1,XLOC_022355,MSTRG.16757.1,1.026873,0.999814,1.130154,1.115598,0.557321,2.703709,2.651492,0.660827,0.000000,0.207575
226,0.85,3L,2712437,2712929,TCONS_00031592,1,XLOC_015890,MSTRG.10274.1,1.948808,0.994009,1.577002,1.270875,1.390507,3.768191,2.690342,1.034692,0.332947,0.245331
422,0.84,3R,20356140,20356898,TCONS_00045157,1,XLOC_022774,MSTRG.18321.1,6.499248,0.994009,3.655635,3.751255,4.865778,8.146936,8.631846,0.059422,0.229829,0.000000
331,0.84,3L,20036187,20037352,TCONS_00032497,1,XLOC_016795,MSTRG.13430.1,3.470075,0.994009,1.775897,1.557853,4.389355,2.602303,2.665926,0.314940,0.284938,0.397921
24,0.84,2L,4171324,4172086,TCONS_00009102,1,XLOC_004457,MSTRG.758.1,1.312294,0.994009,1.496794,1.511124,3.292685,1.658097,3.663003,0.366976,0.000000,0.000000
146,0.84,2R,12237007,12238016,TCONS_00021236,1,XLOC_010774,MSTRG.7240.1,2.744881,0.994009,1.442457,1.225451,3.636914,1.699904,2.172582,0.544053,0.601291,0.000000


In [77]:
for i in list(top_20_prob.index):
    coord = list(top_20_prob[['chr', 'start', 'end']].apply(lambda x: '..'.join(x.values.astype(str)), axis=1))

coord = list(map(lambda x: str.replace(x, '..', ':', 1), coord))
coord


['2R:13026340..13027051',
 'X:11212364..11213061',
 'X:4753265..4754217',
 'X:4206331..4207482',
 '3R:11510551..11511190',
 '3L:2712437..2712929',
 '3R:20356140..20356898',
 '3L:20036187..20037352',
 '2L:4171324..4172086',
 '2R:12237007..12238016',
 '3L:21450858..21451862',
 '3R:20346548..20347328',
 '2R:8532515..8533035',
 '2L:3434822..3435852',
 '3L:20667898..20668479',
 '3L:9033224..9034052',
 '2R:7416645..7417593',
 '3L:7193611..7194472',
 '3L:5127615..5128196',
 '2L:12829560..12830289']

In [78]:
with open(resultsdir/'top_20_prob_coord.txt', 'w') as output:
    for element in coord:
        output.write(element + "\n")

top_20_prob.to_csv(resultsdir/'top_20_prob.csv', index=False)
final.to_csv(resultsdir/'final_results.csv', index=False)